In [155]:
import os
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input # type: ignore
from tensorflow.keras.preprocessing.image import load_img, img_to_array # type: ignore
from tensorflow.keras.models import Model # type: ignore
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy.io import loadmat
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import median_absolute_error
from sklearn.datasets import fetch_openml
from sklearn.impute import SimpleImputer
import re
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing import image # type: ignore

In [156]:
split_df =  pd.read_csv('split_df.csv',encoding='utf-8-sig')

In [157]:
# load the model
model = VGG16()
# restructure the model
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
# summary
print(model.summary())

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [61]:
# for layer in model.layers[:15]: # مشان نلغي اول 15 طبقة
#     layer.trainable = False

In [158]:
# features = {}
# directory = 'Data/'
def extract_orb_features(image_path):
# for image in os.listdir(directory):
#     image_path = directory+image
    
    # load the image
    img = load_img(image_path, target_size=(224, 224))
    # convert pixel to numpy array
    img = img_to_array(img)
    # reshape the image for the model
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    # preprocess the image
    img = preprocess_input(img)
    # extract features
    feature = model.predict(img, verbose=0)
    # store feature
    # features[image_path] = feature
    return feature.flatten()

In [160]:
split_df = split_df.dropna(subset=['Image'])

In [84]:
model = VGG16(weights='imagenet', include_top=False)

def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

split_df['VGG16_features'] = list(split_df['Image'].apply(extract_features))

1/1 [==============================] - 0s 117ms/step


In [85]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [161]:
split_df['VGG16_features'] = list(split_df['Image'].apply(extract_orb_features))

In [66]:
# feature.tolist()
# features['Data/5725_HASAN_MARAE Optic nerve.png_ left.png']
# store the dictionary using pickle

In [265]:
split_df=split_df.drop(['Table 1','Gender','id','Name','Eye','Age','Spectacles','Image'],axis=1)
# split_df=split_df.dropna(thresh=len(split_df.columns) - 1)

KeyError: "['Table 1', 'Gender', 'id', 'Name', 'Eye', 'Age', 'Spectacles', 'Image'] not found in axis"

In [266]:
X_data = split_df.drop(['Diagnosis'],axis=1)
y_data = split_df['Diagnosis']

In [267]:
X_data['VGG16_features'] = X_data['VGG16_features'].apply(lambda x: np.sum(x))

In [268]:
X_data

,VA,CDR,IOP,VGG16_features
0,0.6,0.5,14.0,2182.982178
1,0.7,0.9,23.0,2182.982178
2,NaN,NaN,25.0,2446.085205
3,9.0,0.8,40.0,2446.085205
6,0.2,0.7,10.0,2685.752197
...,...,...,...,...
85,NaN,NaN,NaN,2640.070801
86,0.3,0.5,15.0,2469.391113
87,0.6,0.7,16.0,2469.391113
88,0.5,0.3,15.0,2839.579834


In [269]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42, shuffle=True)


In [270]:
X_train.dtypes

VA                float64
CDR               float64
IOP               float64
VGG16_features    float32
dtype: object

In [271]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [272]:
X_train_scaled

array([[-0.39805815,  0.03941426,  0.20873827,  0.73502255],
       [-0.58110678, -0.35472838, -1.03499393,  0.15577722],
       [-0.23534825, -0.35472838, -0.24352798, -1.58185657],
       [-0.27602572, -0.27589985, -0.46966111, -1.15703022],
       [ 3.05952721,  0.11824279, -0.58272768,  0.85750735],
       [-0.19467077, -0.03941426, -0.80886081, -0.04624358],
       [-0.23534825,  0.03941426, -0.92192737,  0.74886107],
       [-0.27602572,  0.11824279,  0.32180484,  0.95074706],
       [-0.56076805, -0.03941426, -0.46966111,  1.40613537],
       [-0.23534825, -0.35472838,  0.32180484, -1.61446621],
       [-0.27602572, -0.03941426, -0.35659455, -1.15703022],
       [-0.39805815, -0.11824279, -0.13046142, -2.23427775],
       [-0.27602572, -0.35472838, -0.46966111, -0.15907179],
       [-0.4794131 , -0.03941426,  0.54793796, -0.15907179],
       [-0.56076805,  0.03941426,  2.92233581,  0.65981037],
       [-0.19467077, -0.19707132, -0.69579424,  0.14198128],
       [-0.39805815, -0.

In [273]:
print(np.isnan(X_train_scaled).sum())

20


In [274]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_train_scaled = imputer.fit_transform(X_train_scaled)
X_test_scaled = imputer.transform(X_test_scaled)


In [275]:
clf = svm.SVC(C=0.9,kernel='linear')

clf.fit(X_train_scaled,y_train)

y_pred = clf.predict(X_test_scaled)

# حساب الدقة
accuracy = accuracy_score(y_test, y_pred)

# طباعة الدقة
print(f'accuracy: {accuracy}')

accuracy: 0.8333333333333334


In [276]:
# clf = svm.SVC(C=0.9,kernel='rbf',gamma=2)
clf.set_params(C=1.2,kernel='rbf',gamma=1)## Gamma مدى الليونة

clf.fit(X_train_scaled,y_train)

y_pred = clf.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)

print(f'accuracy: {accuracy}')

accuracy: 0.8888888888888888


In [280]:
# clf = svm.SVC(C=1.0,kernel='poly',degree= 3,gamma=3)
clf.set_params(C=0.9,kernel='poly',degree=1,gamma=40)

clf.fit(X_train_scaled,y_train)

y_pred = clf.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)

print(f'accuracy: {accuracy}')

accuracy: 0.7222222222222222


In [278]:
clf2 = svm.SVC()

clf2.fit(X_train_scaled,y_train)

y_pred = clf2.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)

print(f'accuracy: {accuracy}')

accuracy: 0.6111111111111112
